# Acquire Zillow

## For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

## You will have a zillow.ipynb file and a helper file for each section in the pipeline.

* Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe.

* Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

* Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

* Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
* Only include properties that include a latitude and longitude value.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

from env import user, host, password
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_connection(database, user=user, host=host, password=password):
    '''get URL with user, host, and password from env '''
    
    return f"mysql+pymysql://{user}:{password}@{host}/{database}"

In [ ]:
def get_sql_data(database,query):
    ''' 
        Take in a database and query
        check if csv exists for the queried database
        if it does read from the csv
        if it does not create the csv then read from the csv  
    '''
    
    if os.path.isfile(f'{database}_query.csv') == False:   # check for the file
        
        df = pd.read_sql(query, get_connection(database))  # create df for query
        
        df.to_csv(f'{database}_query.csv',index = False)   # cache file
        
    return pd.read_csv(f'{database}_query.csv') # return contents of file

In [ ]:
query = '''

select * 
from predictions_2017

left join properties_2017 using(parcelid)
left join airconditioningtype using(airconditioningtypeid)
left join architecturalstyletype using(architecturalstyletypeid)
left join buildingclasstype using(buildingclasstypeid)
left join heatingorsystemtype using(heatingorsystemtypeid)
left join propertylandusetype using(propertylandusetypeid)
left join storytype using(storytypeid)
left join typeconstructiontype using(typeconstructiontypeid)

where latitude is not null

and longitude is not null

'''

database = "zillow"

In [ ]:
df = get_sql_data(database,query)
df.head()

In [ ]:
df.shape

In [ ]:
# drop duplicate parcelid keeping the latest one by transaction date
df = df.sort_values('transactiondate').drop_duplicates('parcelid',keep='last')

In [ ]:
df.shape

In [ ]:
df.parcelid.value_counts()

# Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# get value counts for discrete variables

desc_cols = [col for col in df.columns if (df[col].dtype == "object")]


for col in desc_cols:
    
    print(col)
    print(df[col].value_counts())
    print()

In [ ]:
# distribution of the data
con_cols = [col for col in df.columns if (df[col].dtype == 'int64') | (df[col].dtype == 'float64')]

for col in con_cols:
    plt.hist(df[col])
    plt.title(f"{col} distribution")
    plt.show()

# Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
df.isnull().sum() # get columns paired with the number of nulls in that column

In [ ]:
df.isnull().sum()/df.shape[0] # get percent of nulls in each column

In [ ]:
# capturing values in variables

num_missing = df.isnull().sum()
pct_missing = df.isnull().sum()/df.shape[0] 

In [ ]:
pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}) # create dataframe using variables

In [ ]:
def nulls_by_col(df):
    '''
    take in a dataframe 
    return a dataframe with each cloumn name as a row 
    each row will show the number and percent of nulls in the column
    
    '''
    
    num_missing = df.isnull().sum()   # get columns paired with the number of nulls in that column
    
    pct_missing = df.isnull().sum()/df.shape[0] # get percent of nulls in each column
    
    return pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}) # create/return dataframe

In [ ]:
nulls_by_col(df)

### Takeaways: 
* Columns have a largely varied number of nulls 
* Drop columns with less than half non_null values
* Treat the rest as a case by case

# Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.


In [ ]:
df.isnull().sum(axis=1) # number of columns that are missing in each row

In [ ]:
df.isnull().sum(axis=1)/df.shape[1]*100 # percent of columns missing in each row 

In [ ]:
# capture series in veriables

num_cols_missing = df.isnull().sum(axis=1)

pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100

In [ ]:
# create a dataframe for the series and reset the index creating an index column

df_cols = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index()
df_cols

In [ ]:
# group by count of both columns, turns index column into a count of matching rows

df_cols = df_cols.groupby(['num_cols_missing','pct_cols_missing']).count()
df_cols.head()

In [ ]:
# change the index name and reset the index

df_cols = df_cols.rename(index=str, columns={'index': 'num_rows'}).reset_index()
df_cols.head()

In [ ]:
def nulls_by_row(df):
    '''take in a dataframe 
       get count of missing columns per row
       percent of missing columns per row 
       and number of rows missing the same number of columns
       in a dataframe'''
    
    num_cols_missing = df.isnull().sum(axis=1) # number of columns that are missing in each row
    
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100  # percent of columns missing in each row 
    
    # create a dataframe for the series and reset the index creating an index column
    # group by count of both columns, turns index column into a count of matching rows
    # change the index name and reset the index
    
    return (pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index()
            .groupby(['num_cols_missing','pct_cols_missing']).count()
            .rename(index=str, columns={'index': 'num_rows'}).reset_index())

In [ ]:
nulls_by_row(df)

### Takeaways: 
* Rows missing between 33.33 and 70 percent of cols
* Drop rows with less than half non_null cols
* Treat the rest as a case by case

# Prepare

## Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
df.propertylandusedesc.value_counts()

In [ ]:
# remove rows based on propertylanduse
df = df[(df.propertylandusedesc == 'Single Family Residential') |
          (df.propertylandusedesc == 'Mobile Home') |
          (df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') |
          (df.propertylandusedesc == 'Townhouse')]

In [ ]:
df.shape

In [ ]:
# median area 2017: 2,400

df.boxplot(column='calculatedfinishedsquarefeet')

In [ ]:
df.boxplot(column='bedroomcnt')

In [ ]:
df.boxplot(column='bathroomcnt')

In [ ]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[f'{col}'].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]
        
    return df

In [ ]:
df = remove_outliers(df, 1.5, ['calculatedfinishedsquarefeet', 'bedroomcnt', 'bathroomcnt'])

In [ ]:
df.shape

# Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

## The input:
* A dataframe
* A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
* A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

## The output:
* The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

In [ ]:
# set test proportions
prop_required_column = .5
prop_required_row = .5

In [ ]:
int(round(prop_required_column*df.shape[0],0)) # get minimum acceptable nulls in each column

In [ ]:
# set as thresh

col_thresh = int(round(prop_required_column*df.shape[0],0))

In [ ]:
# drop columns based on threshold number
df.dropna(axis=1, thresh=col_thresh, inplace=True)

In [ ]:
# same for rows 

row_thresh = int(round(prop_required_row*df.shape[1],0))

In [ ]:
df.dropna(axis=0, thresh=row_thresh, inplace=True)

In [ ]:
df.shape

In [ ]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .5):
    ''' 
        take in a dataframe and a proportion for columns and rows
        return dataframe with columns and rows not meeting proportions dropped
    '''
    col_thresh = int(round(prop_required_column*df.shape[0],0)) # calc column threshold
    
    df.dropna(axis=1, thresh=col_thresh, inplace=True) # drop columns with non-nulls less than threshold
    
    row_thresh = int(round(prop_required_row*df.shape[1],0))  # calc row threshhold
    
    df.dropna(axis=0, thresh=row_thresh, inplace=True) # drop columns with non-nulls less than threshold
    
    return df

# Decide how to handle the remaining missing values:

* Fill with constant value.
* Impute with mean, median, mode.
* Drop row/column

In [ ]:
df.isnull().sum()[df.isnull().sum()>0] # look at cols with nulls

In [ ]:
df.isnull().sum()[df.isnull().sum()>16000] # look at cols with large amounts of nulls

In [ ]:
# value counts for cals with large amounts of nulls
for col in ['heatingorsystemtypeid', 'buildingqualitytypeid', 'propertyzoningdesc', 'unitcnt', 'heatingorsystemdesc']:
    
    print(col)
    print(df[col].value_counts())

In [ ]:
# dropping the columns with 16K missing values too much to fill/impute/drop rows

df = df.drop(columns=['heatingorsystemtypeid', 'buildingqualitytypeid', 'propertyzoningdesc', 'unitcnt', 'heatingorsystemdesc'])

In [ ]:
df.shape

In [ ]:
df.isnull().sum()[df.isnull().sum()>0] # look at remaining nulls

In [ ]:
# value counts for remaining cols that are descreet

for col in ['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'yearbuilt', 'censustractandblock']:
    
    print(col)
    print(df[col].value_counts())
    print()

In [ ]:
# impute columns *do this after you split*

def impute(df, my_strategy, column_list):
    ''' take in a df strategy and cloumn list
        return df with listed columns imputed using input stratagy
    '''
        
    imputer = SimpleImputer(strategy=my_strategy)  # build imputer

    df[column_list] = imputer.fit_transform(df[column_list]) # fit/transform selected columns

    return df

In [ ]:
df = impute(df, 'most_frequent', ['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'yearbuilt', 'censustractandblock'])

In [ ]:
df.isnull().sum()[df.isnull().sum()>0] # look at remaining nulls

In [ ]:
df = impute(df, 'median', ['finishedsquarefeet12', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount'])

In [ ]:
df.isnull().sum()

# wrangle_zillow

In [ ]:
###############acquire#################


def get_connection(database, user=user, host=host, password=password):
    '''get URL with user, host, and password from env '''
    
    return f"mysql+pymysql://{user}:{password}@{host}/{database}"
    
    
def cache_sql_data(df, database):
    '''write dataframe to csv with title database_query.csv'''
    
    df.to_csv(f'{database}_query.csv',index = False)
        

def get_sql_data(database,query):
    ''' check if csv exists for the queried database
        if it does read from the csv
        if it does not create the csv then read from the csv  
    '''
    
    if os.path.isfile(f'{database}_query.csv') == False:   # check for the file
        
        df = pd.read_sql(query, get_connection(database))  # create file 
        
        cache_sql_data(df, database) # cache file
        
    return pd.read_csv(f'{database}_query.csv') # return contents of file


def get_zillow_data():
    ''' acquire zillow data'''
    
    query = '''

    select * 
    from predictions_2017

    left join properties_2017 using(parcelid)
    left join airconditioningtype using(airconditioningtypeid)
    left join architecturalstyletype using(architecturalstyletypeid)
    left join buildingclasstype using(buildingclasstypeid)
    left join heatingorsystemtype using(heatingorsystemtypeid)
    left join propertylandusetype using(propertylandusetypeid)
    left join storytype using(storytypeid)
    left join typeconstructiontype using(typeconstructiontypeid)

    where latitude is not null

    and longitude is not null

    '''

    database = "zillow"
    
    df = get_sql_data(database,query) # create/read csv for query
    
    df = df.sort_values('transactiondate').drop_duplicates('parcelid',keep='last') # drop duplicate parcelids keeping the latest
    
    return df 

#################################prepare###############################

def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[f'{col}'].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        return df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]  
    

def handle_missing_values(df, prop_required_column = .5, prop_required_row = .5):
    ''' 
        take in a dataframe and a proportion for columns and rows
        return dataframe with columns and rows not meeting proportions dropped
    '''
    col_thresh = int(round(prop_required_column*df.shape[0],0)) # calc column threshold
    
    df.dropna(axis=1, thresh=col_thresh, inplace=True) # drop columns with non-nulls less than threshold
    
    row_thresh = int(round(prop_required_row*df.shape[1],0))  # calc row threshhold
    
    df.dropna(axis=0, thresh=row_thresh, inplace=True) # drop columns with non-nulls less than threshold
    
    return df    
    
    
def impute(df, my_strategy, column_list):
    ''' take in a df strategy and cloumn list
        return df with listed columns imputed using input stratagy
    '''
        
    imputer = SimpleImputer(strategy=my_strategy)  # build imputer

    df[column_list] = imputer.fit_transform(df[column_list]) # fit/transform selected columns

    return df

def prepare_zillow(df):
    ''' Prepare Zillow Data'''
    
    # Restrict propertylandusedesc to those of single unit
    df = df[(df.propertylandusedesc == 'Single Family Residential') |
          (df.propertylandusedesc == 'Mobile Home') |
          (df.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') |
          (df.propertylandusedesc == 'Townhouse')]
    
    # remove outliers in bed count, bath count, and area to better target single unit properties
    df = remove_outliers(df, 1.5, ['calculatedfinishedsquarefeet', 'bedroomcnt', 'bathroomcnt'])
    
    # dropping cols/rows where more than half of the values are null
    df = handle_missing_values(df, prop_required_column = .5, prop_required_row = .5)
    
    # dropping the columns with 17K missing values too much to fill/impute/drop rows
    df = df.drop(columns=['heatingorsystemtypeid', 'buildingqualitytypeid', 'propertyzoningdesc', 'unitcnt', 'heatingorsystemdesc'])
    
    # imputing descreet columns with most frequent value
    df = impute(df, 'most_frequent', ['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'yearbuilt', 'censustractandblock'])
    
    # imputing continuous columns with median value
    df = impute(df, 'median', ['finishedsquarefeet12', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount'])
    
    return df

In [ ]:
df = prepare_zillow(get_zillow_data())
df.head()